In [1]:
!pip install invisible_watermark transformers safetensors
!pip install diffusers==0.18.0

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
  Using cached https://mirrors.aliyun.com/pypi/packages/af/0c/ac0d96b7cdb3168b71758f0a3b600fa3ca725f1d6bf2121acf1b62ec868f/diffusers-0.18.0-py3-none-any.whl (1.2 MB)
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.28.0.dev0
    Uninstalling diffusers-0.28.0.dev0:
      Successfully uninstalled diffusers-0.28.0.dev0


In [4]:
from diffusers import DiffusionPipeline
import torch

pipe = DiffusionPipeline.from_pretrained("/ssdshare/LLMs/stable-diffusion-xl-base-1.0/", torch_dtype=torch.float16, use_safetensors=True, variant="fp16")
pipe.to("cuda")

# if using torch < 2.0
# pipe.enable_xformers_memory_efficient_attention()

The config attributes {'force_upcast': True} were passed to AutoencoderKL, but are not expected and will be ignored. Please verify your config.json configuration file.


StableDiffusionXLPipeline {
  "_class_name": "StableDiffusionXLPipeline",
  "_diffusers_version": "0.18.0",
  "force_zeros_for_empty_prompt": true,
  "scheduler": [
    "diffusers",
    "EulerDiscreteScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "text_encoder_2": [
    "transformers",
    "CLIPTextModelWithProjection"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "tokenizer_2": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [5]:
prompt = "An astronaut riding a green horse"
images = pipe(prompt=prompt).images[0]
print(images)

  0%|          | 0/50 [00:00<?, ?it/s]

<PIL.Image.Image image mode=RGB size=1024x1024 at 0x7F4480D2D900>


In [7]:
from PIL import Image
import os

# Create the directory if it doesn't exist
os.makedirs("generate_example", exist_ok=True)

# Save the image

images.save("generate_example/1.png")

In [12]:
from diffusers import DiffusionPipeline
import torch


# load both base & refiner
base = DiffusionPipeline.from_pretrained(
    "/ssdshare/LLMs/stable-diffusion-xl-base-1.0/", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
)
base.to("cuda")
refiner = DiffusionPipeline.from_pretrained(
    "/ssdshare/LLMs/stable-diffusion-xl-refiner-1.0/",
    text_encoder_2=base.text_encoder_2,
    vae=base.vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
)
refiner.to("cuda")

# Define how many steps and what % of steps to be run on each experts (80/20) here
n_steps = 40
high_noise_frac = 0.8

prompt = "A majestic lion jumping from a big stone at night"

# run both experts
image_base = base(
    prompt=prompt,
    num_inference_steps=n_steps,
    # denoising_end=high_noise_frac,
    output_type="latent",
).images

image = refiner(
    prompt=prompt,
    num_inference_steps=n_steps,
    # denoising_start=high_noise_frac,
    image=image_base,
).images[0]
image.save("generate_example/refined.png")


The config attributes {'force_upcast': True} were passed to AutoencoderKL, but are not expected and will be ignored. Please verify your config.json configuration file.


  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

In [13]:
n_steps = 100
high_noise_frac = 0.8

prompt = "Jay Chou playing the piano in a concert, with realistic style"

# run both experts
image_base = base(
    prompt=prompt,
    num_inference_steps=n_steps,
    # denoising_end=high_noise_frac,
    output_type="latent",
).images

image = refiner(
    prompt=prompt,
    num_inference_steps=n_steps,
    # denoising_start=high_noise_frac,
    image=image_base,
).images[0]
image.save("generate_example/person_real.png")

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

In [15]:
from diffusers import DiffusionPipeline
import torch


# load both base & refiner
base = DiffusionPipeline.from_pretrained(
    "/ssdshare/LLMs/stable-diffusion-xl-base-1.0/", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
)
base.to("cuda:1")
refiner = DiffusionPipeline.from_pretrained(
    "/ssdshare/LLMs/stable-diffusion-xl-refiner-1.0/",
    text_encoder_2=base.text_encoder_2,
    vae=base.vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
).to("cuda:1")
n_steps = 50
high_noise_frac = 0.8

prompt = "Jay Chou in a fantasy world, playing piano, 8k resolution, 16:9 aspect ratio, 60fps, with a dreamy aesthetic."
image_base = base(
    prompt=prompt,
    num_inference_steps=n_steps,
    # denoising_end=high_noise_frac,
    output_type="latent",
).images

image = refiner(
    prompt=prompt,
    num_inference_steps=n_steps,
    # denoising_start=high_noise_frac,
    image=image_base,
).images[0]
image.save("generate_example/jay_real.png")

The config attributes {'force_upcast': True} were passed to AutoencoderKL, but are not expected and will be ignored. Please verify your config.json configuration file.


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]